In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
)

chain = prompt | llm

In [4]:
chain.invoke({"num": 5})

AIMessage(content='50입니다.', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-01fcecfc-d72a-477e-ac38-c61a99305586-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19})

In [5]:
chain.invoke(5)

AIMessage(content='50입니다.', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3c097136-6e42-4072-a4d3-189fdab9e484-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19})

In [6]:
from langchain_core.runnables import RunnablePassthrough

# runnable
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

In [7]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

runnable_chain.invoke(10)

AIMessage(content='100입니다.', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-779bd9c0-9154-47e4-a7d6-686015af6c53-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19})

In [8]:
RunnablePassthrough().invoke({"num": 1})

{'num': 1}

In [9]:
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

In [10]:
from langchain_core.runnables import RunnableParallel

runnable = RunnableParallel(
  
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})


{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [11]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [12]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='서울입니다.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 19, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-535ddaa1-4840-443d-bd56-c642c2968eb2-0', usage_metadata={'input_tokens': 19, 'output_tokens': 5, 'total_tokens': 24}),
 'area': AIMessage(content='대한민국의 총 면적은 약 100,363 km² 입니다.', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 20, 'total_tokens': 45}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3657d976-9b62-4303-899c-8ebf2b267756-0', usage_metadata={'input_tokens': 20, 'output_tokens': 25, 'total_tokens': 45})}

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime


def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)

'Jul-05'

In [14]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
print(chain.invoke(3))

물론입니다. 7월 5일이 생일인 몇몇 유명인들을 나열해 보겠습니다:

1. **장 미셸 자르 (Jean-Michel Jarre)** - 1948년 7월 5일
   - 프랑스의 전자 음악 작곡가이자 프로듀서입니다.

2. **에바 그린 (Eva Green)** - 1980년 7월 5일
   - 프랑스의 배우로, 영화 "카지노 로얄"과 "300: 제국의 부활" 등에서 활약했습니다.

3. **로지 헌팅턴 휘틀리 (Rosie Huntington-Whiteley)** - 1987년 7월 5일
   - 영국의 모델이자 배우로, "트랜스포머 3"와 "매드 맥스: 분노의 도로"에 출연했습니다.

이 외에도 7월 5일에 태어난 많은 유명인들이 있습니다.


In [16]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


def length_function(text):
    return len(text)



def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)



def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)


In [17]:
chain.invoke({"word1": "hello", "word2": "world"})


AIMessage(content='5 + 25는 30입니다.', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 22, 'total_tokens': 31}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8ea6f162-5efe-4235-9630-5f7e54831ec1-0', usage_metadata={'input_tokens': 22, 'output_tokens': 9, 'total_tokens': 31})